# Imports e confing

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from dataclasses import dataclass, fields, asdict

import time

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(4)
wait = WebDriverWait(driver, 10)

# Configurar site

In [3]:
driver.get('https://www.transparencia.sc.gov.br/remuneracao-servidores')

### Salvar os cargos de acordo com o send_keys

In [4]:
cargo = driver.find_element(By.CSS_SELECTOR, '#input-cargo')
cargo.clear()
cargo.send_keys('SECRETARIO')

In [20]:
list_element = driver.find_element(By.CSS_SELECTOR, '#filtro-cargo ul.lista-opcoes')
scroll_origin = ScrollOrigin.from_element(list_element)
ActionChains(driver).scroll_from_origin(scroll_origin,0, 10000).perform()

In [21]:
cargos = driver.find_elements(By.CSS_SELECTOR, '#filtro-cargo ul.lista-opcoes > li > a > span:nth-child(2)')
cargos_extraido = [] # Necessario inspecionar a pagina para os melhores resultados
for cargo in cargos:
    cargos_extraido.append(cargo.text)

In [221]:
def buscar_cargo(nome_cargo):
    '''
    Buscar o nome do cargo e clica no cargo
    '''
    try:
        cargo = driver.find_element(By.CSS_SELECTOR, '#input-cargo')
        ActionChains(driver).scroll_to_element(cargo).perform()
        ActionChains(driver).scroll_by_amount(delta_x=0,delta_y=300 ).perform()
        cargo.clear()
        cargo.send_keys(nome_cargo)
        time.sleep(2)
        list_element = driver.find_element(By.CSS_SELECTOR, '#filtro-cargo ul.lista-opcoes > li:last-child')
        
        cargos = driver.find_element(By.CSS_SELECTOR, '#filtro-cargo ul.lista-opcoes > li:last-child > a > span').text
        if 'NÃO INFORMADO' in cargos:
            print('Não existe registros')
            return 'NÃO TEM'
        print(f'Clicando em: {cargos}')
        list_element.click()
        return 'Tem'
    except Exception as e:
        print(f'Erro na busca do cargo: {cargos}', type(e))
        



In [22]:
def buscar_cargo_v2(nome_cargo):
    '''
    Buscar o nome do cargo e clica no cargo
    '''
    try:
        cargo = driver.find_element(By.CSS_SELECTOR, '#input-cargo')
        ActionChains(driver).scroll_to_element(cargo).perform()
        ActionChains(driver).scroll_by_amount(delta_x=0,delta_y=300 ).perform()
        cargo.clear()
        cargo.send_keys(nome_cargo)
        time.sleep(2)
        
        cargos = driver.find_elements(By.CSS_SELECTOR, '#filtro-cargo ul.lista-opcoes > li > a > span')
        for c in cargos:
            if nome_cargo == c.text:
                print(f'Clicando em: {c.text}')
                scroll_origin = ScrollOrigin.from_element(c)
                ActionChains(driver).scroll_from_origin(scroll_origin,0,0).scroll_to_element(c).perform()
                c.click()
                return 'Tem'
        for c in cargos:
            if 'NÃO INFORMADO' in c.text:
                print('Não existe registros')
                return 'NÃO TEM'
        return 'Tem'
    except Exception as e:
        print(f'Erro na busca do cargo: {cargos}', type(e))
        



In [26]:
for c in cargos_extraido:
    print(c)

SECRETARIO DE CONSELHO PENITENCIARIO
SECRETARIO DE ESCOLA
SECRETARIO EXECUTIVO BILINGUE
SECRETARIO GERAL
SECRETARIO DA COMISSAO CENTRAL DE LICENCIAMENTO AMBIENTAL
SECRETARIO DA COMISSAO INTERGESTORES BIPARTITE
SECRETARIO DE CALCULOS E PERICIAS
SECRETARIO DO CONSELHO ESTADUAL DE CULTURA
SECRETARIO DO CONSELHO ESTADUAL DE ESPORTE
SECRETARIO DO CONSELHO ESTADUAL DE SAUDE
SECRETARIO DO CONSELHO PENITENCIARIO
SECRETARIO DO GRUPO GESTOR DE GOVERNO
SECRETARIO DO PROCESSO ADMINISTRATIVO
SECRETARIO DA CAMARA DE GESTAO E SOLUCAO DE CONFLITOS
SECRETARIO DAS COMISSOES PERMANENTES FUNCIONAIS
SECRETARIO DE CONSELHO
SECRETARIO EXECUTIVO DE ARTICULACAO NACIONAL
SECRETARIO EXECUTIVO DO CONSELHO ESTADUAL DE TRANSITO (CETRAN)
SECRETARIO ADJUNTO DA COMUNICACAO
SECRETARIO ADJUNTO DA EDUCACAO
SECRETARIO ADJUNTO DA FAZENDA
SECRETARIO ADJUNTO DA INFRAESTRUTURA E MOBILIDADE
SECRETARIO ADJUNTO DA SAUDE
SECRETARIO ADJUNTO
SECRETARIO ADJUNTO DA CASA CIVIL
SECRETARIO ADJUNTO DA PROTECAO E DEFESA CIVIL
SECRETARIO A

In [25]:
cargos_extraido.pop(0)

'ASSISTENTE PESSOAL SECRETARIO'

### Funcoes

In [13]:
def pegar_orgao(palavra_chave):
    '''
    Esta função de acordo com a palavra chave específicada retorna uma lista
    ele verifica se a palavra contém nas seleções e retorna todos os orgãos 
    correspondentes. 
    '''
    orgao_origem = []
    #Localizo o Select que contém todos os órgãos de origem
    for n in driver.find_element(By.CSS_SELECTOR, '#select-orgao-origem').get_property('children'): 
        if f'{palavra_chave}' in n.text:
            orgao_origem.append(n.text)
        
    return orgao_origem

In [27]:
def quantidade_mes():
    '''
    Retorna a quantidade de meses
    '''
    mes = driver.find_element(By.CSS_SELECTOR, '#select-mes').get_property('children')
    return len(mes)

In [28]:
def selecionar_mes(index):
    '''
    Seleciona o mês de acordo com o index e retorna qual mês foi selecionado
    '''
    mes = Select(driver.find_element(By.CSS_SELECTOR, '#select-mes'))
    mes.select_by_index(index)
    mes_atual = driver.find_element(By.CSS_SELECTOR, '#select-mes').get_property('children')[index].text
    return mes_atual

In [29]:
def selecionar_ano(index):
    '''
    Seleciona o ano de acordo com a string específicada ex: "2025"
    '''
    ano = Select(driver.find_element(By.CSS_SELECTOR, '#select-ano'))
    ano.select_by_visible_text(f'{index}')

In [30]:
def retornar_linhas_tabela():
    '''
    Retorna todas as linhas da tabela
    '''
    linhas = driver.find_elements(By.CSS_SELECTOR, '#tabela-servidores > tbody > tr')
    link = linhas[0].get_property('children')[-1].find_element(By.CSS_SELECTOR, 'a') # Procura o link do holerite detalhado
    return linhas

In [31]:
from random import uniform

def time_random(a,b):
    '''
    A = número inicial
    B = Número final
    Retorna numero quebrado entre esses números
    '''
    numero = round(uniform(a,b),2)
    print(f'Pausa de {numero} segundos!')
    espera = time.sleep(numero)
    return espera

In [32]:
def mudar_aba(i):
    abas = driver.window_handles
    driver.switch_to.window(abas[i])

### Funçao para extraçao de dados e testes

In [54]:
@dataclass
class TabelaServidor:
    referencia:str
    nome:str
    cargo_comissionado:str
    orgao_origem:str
    forma_de_ingresso:str
    categoria:str
    data_nomeacao_contratacao:str
    data_exoneracao:str
    renumeracao_basica_1:str
    pagamento_retroativo_2:str
    gratificacoes_eventuais_3:str
    verbas_indenizatorias_4:str
    ferias_5:str
    _13_salario_6:str
    outros_proventos_7:str
    deducoes_obrigatorios_8:str
    descontos_faltas_restricoes_estado:str
    adiamento_13_salario:str
    renumeracao_bruta:str
    renumeracao_liquida:str

def extrair_cargo_comissionado_2_dataclass():


    try:
        # Pega os dois possíveis quadros eu seleciona apenas o que seja comissionado
        quadro_cargos = driver.find_elements(By.CSS_SELECTOR, 'div.quadro-cargo')
        quadro_cargo = []
        for q_cargo in quadro_cargos:
            print(q_cargo.find_element(By.CSS_SELECTOR, 'div.conteudo-documento-cabecalho-texto').text)
            if 'CARGO COMISSIONADO' in q_cargo.find_element(By.CSS_SELECTOR, 'div.conteudo-documento-cabecalho-texto').text:
                quadro_cargo.append(q_cargo)
    except Exception as e:
        print('Erro acho achar o quadro cargo erro: ',type(e))
        
    try:
        # Do quadro selecionado capturo os dados de acordo com o index dele que armazenei numa lista
        quadro_cargo_selecionado = quadro_cargo[0]
        conteudos = quadro_cargo_selecionado.find_elements(By.CSS_SELECTOR, 'div.conteudo-documento-texto1')

        cargo_comissionado= conteudos[0].text
        orgao_origem= conteudos[1].text
        forma_de_ingresso= conteudos[5].text
        categoria= conteudos[8].text
        data_nomeacao_contratacao= conteudos[11].text
        data_exoneracao= conteudos[14].text
    except Exception as e:
        print('Erro ao capturar dados do quadro_comissionado ',type(e))
        
        
    try:
        quadro_renumeracaos = driver.find_elements(By.CSS_SELECTOR, 'div.bloco-remuneracao')
        quadro_renumeracao = []
        for q_renu in quadro_renumeracaos:
            texto = q_renu.find_element(By.CSS_SELECTOR, 'div.conteudo-documento-linha div.conteudo-documento-cabecalho-texto').text
            texto.strip().upper()
            orgao_origem.strip().upper()
            if orgao_origem in texto:
                quadro_renumeracao.append(q_renu)
            
        try:
            quadro_renu_selecionado = quadro_renumeracao[0]
            valor = quadro_renu_selecionado.find_elements(By.CSS_SELECTOR, 'div.conteudo-documento-texto3')
            info_valor = quadro_renu_selecionado.find_elements(By.CSS_SELECTOR, 'div.conteudo-documento-label3')
            desconto = quadro_renu_selecionado.find_elements(By.CSS_SELECTOR, 'div.conteudo-documento-texto4')
            info_desconto = quadro_renu_selecionado.find_elements(By.CSS_SELECTOR, 'div.conteudo-documento-label4')
            renumeracao_basica = valor[1].text
            renumeracao_retroativa = valor[2].text
            gratificacoes_eventuais = valor[3].text
            verbas_indenizatorias = valor[4].text
            ferias = valor[5].text
            decimo_terceiro = valor[6].text
            outros_proventos = valor[7].text
            deducoes_obrigatorios = desconto[1].text
            desconto_faltas_res = desconto[2].text
            adiamento_decimo = desconto[3].text
            renumeracao_bruta = valor[0].text
            renumeracao_liquida = desconto[6].text
        except Exception as e:
            print(f'Erro na captura de dados da renumeração erro: {type(e)}')
            print('Prosseguindo o codigo')
            pass
        else:
            try:
                referencia = driver.find_elements(By.CSS_SELECTOR, '#area-detalhamento h3.conteudo-documento-titulo > span')[1].text.split(':')[1].strip()
                nome = driver.find_element(By.CSS_SELECTOR, '#area-detalhamento > div > div > div:nth-child(1) > h3 > span').text
            except Exception as e:
                print('Erro ao pegar referencia')
            try:
                
                registro = TabelaServidor(
                    referencia=referencia,
                    nome=nome,
                    cargo_comissionado=cargo_comissionado,
                    orgao_origem=orgao_origem,
                    forma_de_ingresso=forma_de_ingresso,
                    categoria=categoria,
                    data_nomeacao_contratacao=data_nomeacao_contratacao,
                    data_exoneracao=data_exoneracao,
                    renumeracao_basica_1=renumeracao_basica,
                    pagamento_retroativo_2=renumeracao_retroativa,
                    gratificacoes_eventuais_3=gratificacoes_eventuais,
                    verbas_indenizatorias_4=verbas_indenizatorias,
                    ferias_5=ferias,
                    _13_salario_6=decimo_terceiro,
                    outros_proventos_7=outros_proventos,
                    deducoes_obrigatorios_8=deducoes_obrigatorios,
                    descontos_faltas_restricoes_estado=desconto_faltas_res,
                    adiamento_13_salario=adiamento_decimo,
                    renumeracao_bruta=renumeracao_bruta,
                    renumeracao_liquida=renumeracao_liquida
                )
                print('Dados coletados com sucesso!')
                print(registro)
            except Exception as e:
                print('Erro dataclass', e)
                
                
    except Exception as e:
        print('Não existe holerite comissionado')
        pass
    return registro

In [136]:
for i in range(0,len(valor)):
    print(f'INDEX: {i} {info_valor[i].text}: {valor[i].text}')


INDEX: 0 Remuneração Bruta: R$  25.322,25
INDEX: 1 Remuneração Básica (1): R$  25.322,25
INDEX: 2 Pagamento Retroativo (2): R$  0,00
INDEX: 3 Gratificações Eventuais (3): R$  0,00
INDEX: 4 Verbas Indenizatórias (4): R$  0,00
INDEX: 5 Férias (5): R$  0,00
INDEX: 6 13º Salário (6): R$  0,00
INDEX: 7 Outros Proventos (7): R$  0,00


In [24]:
for i in range(0,len(conteudos)):
    print(F'INDEX: {i}', f': {conteudos[i].text}')


NameError: name 'conteudos' is not defined

# Extração 

In [34]:
len(cargos_extraido)

68

In [35]:
mudar_aba(0)

Abaixo vou pegar dados secretarios que o orgao de origem dele comissionado seja o mesmo do contracheque

In [36]:
ano_atual = 2020
ano_final = 2023


dados_nao_encontrados = []
registros = []
try:
    while ano_atual >= 2020:
        
        selecionar_ano(f'{ano_atual}')
        print(f'Selecionando ano: {ano_atual}')
        
        qtd_mes = quantidade_mes()
        print(f'{qtd_mes} Meses Disponíveis')
        
        for i in range(0,qtd_mes): # Iterando sobre a quantidade de meses e usando o index para selecionar
            mes = selecionar_mes(i)
            print('='*10)
            print(f'Selecionando mês: {mes}/{ano_atual}')
            print('='*10)
            # Uso a lista de cargos_extraido para digitar:
            
            for cargo_ex in cargos_extraido:
                
                try:
                    time_random(0.2,0.3)
                    print(f'Buscando por: {cargo_ex}')
                    cargo_buscar = buscar_cargo_v2(cargo_ex)
                    
                    if 'NÃO TEM' in cargo_buscar:
                        continue
                    
                    try:
                        time_random(1.2,1.5)
                        print('Retornando linhas')
                        linhas = driver.find_elements(By.CSS_SELECTOR, '#tabela-servidores > tbody > tr')
                        time_random(0.2,0.4)
                        qtd_linhas = len(linhas)
                        print('Total de linhas:', qtd_linhas)
                        if qtd_linhas == 0:
                            print(f'Não existe registro no site para o: {cargo_ex}')
                            dados ={
                                'Cargo':cargo_ex,
                                'Data': f'{mes}/{ano_atual}',
                                'Situação' : 'Sem registros'
                            }
                            dados_nao_encontrados.append(dados)
                            print('Continuando para o próximo')
                            continue
                        for linha in linhas:
                            link = linha.get_property('children')[-1].find_element(By.CSS_SELECTOR, 'a') # Procura o link do holerite detalhado
                            ActionChains(driver).scroll_to_element(link).perform()
                            link.click()
                            time_random(0.2,0.7)
                            mudar_aba(-1)
                            print('Aba aberta!')
                            time.sleep(0.2)
                            try: 
                                registro = extrair_cargo_comissionado_2_dataclass()
                                registros.append(registro)
                            except Exception as e:
                                print('Não é comissionado ou não holerites comissionado')
                                dados ={
                                    'Cargo':cargo_ex,
                                    'Data': f'{mes}/{ano_atual}',
                                    'Situação' : 'Não há dados comissionados'
                                }
                                dados_nao_encontrados.append(dados)
                            finally:
                                driver.close()
                                mudar_aba(0)
                    except Exception as e:
                        print(f'Não existe registro no site para o: {cargo_ex}', e)
                        dados ={
                            'Cargo':cargo_ex,
                            'Data': f'{mes}/{ano_atual}',
                            'Situação' : 'Sem registros'
                        }
                        dados_nao_encontrados.append(dados)
                        
                except Exception as e:
                    print(f'Erro ao buscar o cargo {cargo_ex}')
            
            time_random(1,1.3)
        
        time_random(1,2)
        
        ano_atual -= 1
except Exception as e:
    print(type(e), e)
finally:
    df = pd.DataFrame(registros)
    df_erros = pd.DataFrame(dados_nao_encontrados)


Selecionando ano: 2020
12 Meses Disponíveis
Selecionando mês: Janeiro/2020
Pausa de 0.23 segundos!
Buscando por: SECRETARIO DE CONSELHO PENITENCIARIO
Clicando em: SECRETARIO DE CONSELHO PENITENCIARIO
Pausa de 1.26 segundos!
Retornando linhas
Pausa de 0.31 segundos!
Total de linhas: 1
Pausa de 0.69 segundos!
Aba aberta!
Não é comissionado ou não holerites comissionado
Pausa de 0.28 segundos!
Buscando por: SECRETARIO DE ESCOLA
Clicando em: SECRETARIO DE ESCOLA
Pausa de 1.5 segundos!
Retornando linhas
Pausa de 0.36 segundos!
Total de linhas: 1
Pausa de 0.27 segundos!
Aba aberta!
Não é comissionado ou não holerites comissionado
Pausa de 0.2 segundos!
Buscando por: SECRETARIO EXECUTIVO BILINGUE
Clicando em: SECRETARIO EXECUTIVO BILINGUE
Pausa de 1.26 segundos!
Retornando linhas
Pausa de 0.25 segundos!
Total de linhas: 2
Pausa de 0.61 segundos!
Aba aberta!
Não é comissionado ou não holerites comissionado
Pausa de 0.62 segundos!
Aba aberta!
Não é comissionado ou não holerites comissionado
Pau

In [37]:
df.to_csv('secreatrios_2020.csv', index=False)

In [38]:
df_erros.to_csv('Erros 2020.csv', index =False)

In [40]:
df_erros_1 = pd.read_csv('./Erros 2021_2025.csv')

In [41]:
dfs = [df_erros,df_erros_1]
dfs_erros = pd.concat(dfs)

In [43]:
dfs_erros.to_csv('erros gerais.csv')

In [44]:
mes = selecionar_mes(1)
print('Mes de ', mes)

Mes de  Fevereiro
